In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

In [4]:
df = pd.read_csv("Red_wine_dataset.csv")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5.0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5.0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5.0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6.0
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.52,0.58,10.5,5.0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6.0
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.52,0.75,11.0,6.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.52,0.71,10.2,5.0


Checking for the missing values

In [5]:
print(df.isnull().sum())

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    1
density                 0
pH                      1
sulphates               0
alcohol                 0
quality                 1
dtype: int64


In [6]:
df = df.dropna()

In [7]:
print(df.isnull().sum())

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


In [8]:
df.shape

(1596, 12)

**Lets make a new column named "Good Quality" if the quality is greater then 7 its 1 and if its less than 7 its 0**

In [9]:
df["good quality"] = [1 if x>= 7 else 0 for x in df["quality"]]
df.sample(10)

<ipython-input-9-f5b206ffd5c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["good quality"] = [1 if x>= 7 else 0 for x in df["quality"]]


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,good quality
1575,7.5,0.520,0.40,2.2,0.060,12.0,20.0,0.99474,3.52,0.64,11.8,6.0,0
76,8.8,0.410,0.64,2.2,0.093,9.0,42.0,0.99860,3.54,0.66,10.5,5.0,0
198,5.4,0.835,0.08,1.2,0.046,13.0,93.0,0.99240,3.52,0.85,13.0,7.0,1
710,10.6,1.025,0.43,2.8,0.080,21.0,84.0,0.99850,3.52,0.57,10.1,5.0,0
1512,6.4,0.790,0.04,2.2,0.061,11.0,17.0,0.99588,3.52,0.65,10.4,6.0,0
1188,6.7,0.640,0.23,2.1,0.080,11.0,119.0,0.99538,3.52,0.70,10.9,5.0,0
330,10.2,0.360,0.64,2.9,0.122,10.0,41.0,0.99800,3.52,0.66,12.5,6.0,0
97,7.0,0.500,0.25,2.0,0.070,3.0,22.0,0.99630,3.25,0.63,9.2,5.0,0
610,8.8,0.240,0.54,2.5,0.083,25.0,57.0,0.99830,3.52,0.54,9.2,5.0,0
1500,7.5,0.725,0.04,1.5,0.076,8.0,15.0,0.99508,3.52,0.53,9.6,5.0,0


In [10]:
x = df.drop(["quality","good quality"],axis =1)
y = df["good quality"]

In [11]:
x

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.52,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.52,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.52,0.71,10.2


In [12]:
y

0       0
1       0
2       0
3       0
4       0
       ..
1594    0
1595    0
1596    0
1597    0
1598    0
Name: good quality, Length: 1596, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=7)

In [32]:
print("x_Train: ",x_train.shape)
print("y_train :",y_train.shape)
print("x_test :",y_test.shape)
print("y_test :",y_test.shape)

x_Train:  (1117, 11)
y_train : (1117,)
x_test : (479,)
y_test : (479,)


**We are using RandomForestClassifier mechine learning algorithm**

In [39]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train,y_train)
x_predict = model.predict(x_test)

In [45]:
from sklearn.metrics import accuracy_score
print("Accuracy score :",accuracy_score(y_test,x_predict))

Accuracy score : 0.8935281837160751


In [46]:
input_data = (7.9,0.6,0.06,1.6,0.069,15,59,0.9964,3.3,0.46,9.4)

input_arr = np.asarray(input_data)

input_arr = input_arr.reshape(1,-1)

prediction = model.predict(input_arr)
print(prediction)

if prediction[0] == 1:
  print("Good Quality Wine")
else:
  print("Bad Quality Wine")

[0]
Bad Quality Wine


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
